In [176]:
%matplotlib inline

In [177]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
import numpy as np
import pandas as pd
import sys
import scipy.stats
import pylab as plt
from IPython import display
import pylab as plt
import glob
from collections import defaultdict

In [179]:
sys.path.append('..')

In [180]:
from vimms.Chemicals import *
from vimms.Chromatograms import *
from vimms.MassSpec import *
from vimms.Controller import *
from vimms.Common import *
from vimms.DataGenerator import *
from vimms.DsDA import *

In [181]:
 set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

In [182]:
# base_dir = '..\\data'
# base_dir = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Trained Models'
base_dir = '/Users/simon/University of Glasgow/Vinny Davies - CLDS Metabolomics Project/Trained Models'

In [183]:
ps = load_obj(os.path.join(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))

In [184]:
hmdb = load_obj(os.path.join(base_dir, 'hmdb_compounds.p'))

In [185]:
out_dir = '/Users/simon/vimms_data'

# Create Initial Chemical

In [186]:
ROI_Sources = ["/Users/simon/vimms_data/beer_t10_simulator_files"]
min_ms1_intensity = 1.75E5
rt_range = [(400, 500)]
mz_range = [(200, 400)]
n_peaks = 50
roi_rt_range = [20, 40]
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_peaks, 1, use_database=True, 
                       fixed_mz=False, roi_rt_range=roi_rt_range)
save_obj(dataset, os.path.join(out_dir, 'BaseDataset/dataset.p'))

Saving <class 'list'> to /Users/simon/vimms_data/BaseDataset/dataset.p


In [187]:
for chem in dataset:
    print(np.abs(chem.chromatogram.min_rt - chem.chromatogram.max_rt))

23.639999999999986
30.91300000000001
34.07000000000005
33.34400000000005
27.58499999999998
30.91192
24.62999999999988
36.65499999999997
36.42000000000007
30.340000000000146
23.268
21.972999999999956
36.011000000000024
20.067999999999984
25.733999999999924
20.24000000000001
24.460000000000036
23.871999999999957
24.733999999999924
25.93599999999998
32.3900000000001
22.5
29.205000000000013
22.774999999999977
28.649
30.10300000000001
31.930000000000064
29.180999999999926
34.6412
26.58100000000001
27.55400000000003
29.246999999999957
23.357999999999947
21.681999999999988
21.232000000000085
20.4762
37.3599999999999
24.817999999999984
23.531000000000006
31.873000000000047
24.039000000000044
27.186000000000035
28.537999999999897
36.95600000000002
38.460000000000036
20.876000000000005
38.54899999999998
37.150000000000006
26.47999999999999
20.381999999999948


# Create Multiple Samples

In [198]:
n_samples = [50,50] # number of files per class
classes = ["class%d" % i for i in range(len(n_samples))] # creates default list of classes
intensity_noise_sd = [1000] # noise on max intensity

In [199]:
classes

['class0', 'class1']

Add intensity changes between different classes

In [200]:
change_probabilities = [0 for i in range(len(n_samples))] # probability of intensity changes between different classes
change_differences_means = [0 for i in range(len(n_samples))] # mean of those intensity changes
change_differences_sds = [0 for i in range(len(n_samples))] # SD of those intensity changes

Add experimental variables (examples in comments)

In [201]:
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

Dropout chemicals from in different classes

In [202]:
dropout_probability = 0.2
dropout_probabilities = [dropout_probability for i in range(len(n_samples))]
# dropout_probabilities = None
# dropout_numbers = 2 # number of chemicals dropped out in each class
dropout_numbers = None

Set save location

In [203]:
save_location = os.path.join(out_dir, 'ChemicalFiles')

In [204]:
multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities, dropout_numbers,
                                     experimental_classes, experimental_probabilitities, experimental_sds, save_location=save_location)

26
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_0.p
21
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_1.p
28
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_2.p
21
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_3.p
25
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_4.p
25
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_5.p
23
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_6.p
30
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_7.p
24
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_8.p
26
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_9.p
21
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_10.p
22
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_11.p
27
Saving <class 'list'> to /Users/simon/vimms_data/ChemicalFiles/sample_1

In [205]:
# Check the number of identical formulas in two objects of the same class
formulas0 = set([str(a.formula) for a in multiple_samples.samples[0]])
formulas1 = set([str(a.formula) for a in multiple_samples.samples[1]])
print(formulas0-formulas1)

total_samples = np.sum(multiple_samples.n_samples)
total_samples

{'C11H11N3O2S', 'C13H18O2', 'C18H27NO3', 'C14H11Cl2NO4', 'C5H6Cl6N2O3', 'C4H7Cl2O4P', 'C18H37NO3', 'C10H12ClN3O3S', 'C15H10O7'}


100

In [206]:
save_obj(multiple_samples.missing_chemicals, os.path.join(out_dir, 'MissingChemicals','missing_chemicals.p'))
multiple_samples.missing_chemicals

Saving <class 'list'> to /Users/simon/vimms_data/MissingChemicals/missing_chemicals.p


[[KnownChemical - 'C15H16Cl3N3O2' rt=409.99 max_intensity=1725954.34,
  KnownChemical - 'C16H28O' rt=431.85 max_intensity=1059507.71,
  KnownChemical - 'C19H18ClN5' rt=423.53 max_intensity=329447.21,
  KnownChemical - 'C17H23NO2' rt=415.68 max_intensity=251532.13,
  KnownChemical - 'C11H13N3O3S' rt=421.16 max_intensity=1558628.97,
  KnownChemical - 'C11H28N4' rt=416.56 max_intensity=2005289.30,
  KnownChemical - 'C13H9NO2S' rt=401.56 max_intensity=239654.60,
  KnownChemical - 'C13H9NO2S' rt=428.33 max_intensity=13229609.72,
  KnownChemical - 'C10H16N2O4' rt=414.90 max_intensity=49733322.17,
  KnownChemical - 'C8H11NO5S' rt=412.33 max_intensity=342886.18,
  KnownChemical - 'C13H11N3O5S2' rt=450.38 max_intensity=267161.99,
  KnownChemical - 'C7H9NO4S' rt=428.11 max_intensity=246202.61,
  KnownChemical - 'C21H29NO' rt=425.12 max_intensity=245703.10,
  KnownChemical - 'C8H12O9' rt=459.26 max_intensity=1531653.32,
  KnownChemical - 'C16H34' rt=427.40 max_intensity=76844746.48],
 [KnownChemi

Run MS1 controller and save out .mzML files

In [207]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]
controllers = defaultdict(list)
controller_to_mzml = {}

mzml_dir = os.path.join(out_dir, 'mzmlFiles')
num_classes = len(n_samples)
sample_idx = 0
for j in range(num_classes):
    num_samples = n_samples[j]
    for i in range(num_samples):
        fname = os.path.join(save_location, 'sample_%d.p' % sample_idx) 
        sample = load_obj(fname)
        sample_idx += 1
        
        mass_spec = IndependentMassSpectrometer(POSITIVE, sample, density=ps.density_estimator)
        mzml_filename = os.path.join(mzml_dir,'sample_id_0_number_%d' % i + '_class_%d.mzML' % j)
        controller = SimpleMs1Controller(mass_spec)
        controller.run(min_rt,max_rt)
        controller.write_mzML('my_analysis', mzml_filename)
        
        controllers[j].append(controller)
        controller_to_mzml[controller] = (j, mzml_filename, )

101.0766999999995it [00:00, 585.06it/s]                         
101.0076000000006it [00:00, 756.03it/s]                         
100.65469999999988it [00:00, 661.05it/s]                        
100.08256000000006it [00:00, 764.81it/s]                        
100.82959000000005it [00:00, 658.77it/s]                         
100.91009999999983it [00:00, 664.08it/s]                        
100.93500000000017it [00:00, 662.31it/s]                        
101.21058000000016it [00:00, 586.26it/s]                        
100.69709999999975it [00:00, 725.90it/s]                       
101.0682000000005it [00:00, 662.61it/s]                         
100.94901999999905it [00:00, 725.71it/s]                        
100.52189999999928it [00:00, 752.98it/s]                        
101.13919999999996it [00:00, 660.83it/s]                        
100.76020000000017it [00:00, 772.24it/s]                       
100.29280000000011it [00:00, 671.33it/s]                        
100.12770000000052it [00:0

Print out the missing peaks

In [ ]:
def get_chem_to_peaks(controller):
    chem_to_peaks = defaultdict(list)
    frag_events = controller.mass_spec.fragmentation_events
    for frag_event in frag_events:
        chem = frag_event.chem
        peaks = frag_event.peaks
        chem_to_peaks[chem].extend(peaks)
    return chem_to_peaks

In [ ]:
for controller, (current_class, mzml_filename) in controller_to_mzml.items():
    controller_peaks = get_chem_to_peaks(controller)
    basename = os.path.basename(mzml_filename)
    front, back = os.path.splitext(mzml_filename)
    outfile = front + '.csv'

    missing_peaks = []            
    for other_class in range(num_classes):
        if current_class == other_class:
            continue

        # get the peaks that are present in current_class but missing in other_class
        missing_chems = multiple_samples.missing_chemicals[other_class]
        for chem in missing_chems:
            peaks = controller_peaks[chem]
            for peak in peaks:
                row = (chem.formula.formula_string, current_class, other_class, peak.mz, peak.rt, peak.intensity)
                missing_peaks.append(row)
    
    # convert to dataframe
    columns = ['formula', 'present_in', 'missing_in', 'mz', 'RT', 'intensity']
    missing_df = pd.DataFrame(missing_peaks, columns=columns)
    missing_df.to_csv(os.path.join(out_dir, 'MissingChemicals', os.path.basename(outfile)))